## Preprocessing scRNA-seq data

### Students notebook

#### Let's get our data ready for analysis!

As we discussed in the coding lecture, scRNA-seq data must undergo several preprocessing steps before being amenable for further analysis. Let's apply these steps on your postdoc data!

In [10]:
# First things first: let's import the relevant libraries and read the data

# packages
import numpy as np
import pandas as pd
import scanpy as sc 

# loading the T-cell data
scder = sc.read_h5ad('./data/scder.h5ad')
data_statistics = (scder.n_obs, scder.n_vars)
print(data_statistics)

(424, 20953)


We start by checking whether all cells have at least 200 expressed genes, as well as whether all genes are expressed in at least 3 cells

In [11]:
# exercise on basic filtering: use scanpy filter_cells and filters_genes functions (both of them in the pp module)
# to filter (a) cells with fewer than 200 expressed genes and (b) genes expressed in fewer than 3 cells.
# Update the 'data_statistics' with the new number of cells and genes

# your code here

# Filter cells
sc.pp.filter_cells(scder, min_genes=200)

# Filter genes
sc.pp.filter_genes(scder, min_cells=3)

# Update data_statistics
data_statistics = (scder.n_obs, scder.n_vars)

print(data_statistics)

(424, 14158)


In [12]:
# tip: recall how to use filter_cells and filter_genes, and use them in this order

No cells were eliminated, however the number of genes was reduced.

We should now ensure that no cell has more than 5% of the total amount of reads from mitochondrial mRNA.

In [13]:
# exercise: compute the percentage of mitochondrial reads.

# your code here

# Identify mitochondrial genes (genes starting with 'MT-')
scder.var['mt'] = scder.var_names.str.startswith('MT-')

# Calculate QC metrics, including mitochondrial percentage
sc.pp.calculate_qc_metrics(scder, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


# exercise: filter out cells possibly affected by contamination
# you should ensure that no cell with more than 5% mt reads is left in the dataset
# update the data_statistics tuple after filtering

# your code here

# Filter out cells with more than 5% mitochondrial reads
scder = scder[scder.obs.pct_counts_mt < 5, :]

# Update data_statistics
data_statistics = (scder.n_obs, scder.n_vars)

# you should end up with a new column in the obs dataset, namely pct_counts_mt
print(scder.obs.pct_counts_mt)

1      4.061594
2      1.964647
3      3.634916
5      4.102948
6      3.885202
         ...   
495    4.789249
496    2.351466
497    2.203728
498    2.925115
499    2.829010
Name: pct_counts_mt, Length: 380, dtype: float32


In [14]:
# tip: Recall how we first identified mitochondrial genes (they start with the letters 'MT-'), 
# and then we computed their prevalence with the calculate_qc_metrics function

In [15]:
# exercise: apply the following preprocessing steps in order, with the suggested settings,
# to the new anndata object `scderp`:
# normalize_total with target_sum=1e4
# log1p
# highly_variable_genes
# store a copy of the current scder inside its own "raw" attribute
# highly_variable_genes
# restrict scder to the most variable genes
# scale with max_value=10

# creating a new andata object. Use scderp from now on:
scderp = scder.copy()

# your code here

# Normalize total counts to 1e4 per cell
sc.pp.normalize_total(scderp, target_sum=1e4)

# Log-transform the data
sc.pp.log1p(scderp)

# Identify highly variable genes
sc.pp.highly_variable_genes(scderp, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Store a copy of the current scderp inside its own "raw" attribute
# This is usually done *before* filtering for highly variable genes if you want the raw, normalized, log-transformed data
# for comparison. However, the prompt places it here.
scderp.raw = scderp

# Restrict scderp to the most variable genes
scderp = scderp[:, scderp.var.highly_variable]

# Scale the data
sc.pp.scale(scderp, max_value=10)


print(data_statistics)

(380, 14158)


In [16]:
# tip: once again, this is a simple subset of scder

We can now start the actual preprocessing. We will apply the standard pipeline that we have already used in the coding lecture.

In [17]:
# exercise: apply the following preprocessing steps in order, with the suggested settings, 
# to the new anndata object `scderp`:
# normalize_total with target_sum=1e4
# log1p 
# highly_variable_genes
# store a copy of the current scder inside its own "raw" attribute
# highly_variable_genes
# restrict scder to the most variable genes
# scale with max_value=10

# creating a new andata object. Use scderp from now on:
scderp = scder.copy()

# your code here


scderp

AnnData object with n_obs × n_vars = 380 × 14158
    obs: 'Unnamed: 0', 'cell.type', 'cytokine.condition', 'donor.id', 'batch.10X', 'nGene', 'nUMI', 'percent.mito', 'S.Score', 'G2M.Score', 'Phase', 'cluster.id', 'effectorness', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [18]:
# tip: this is exactly the pipeline we followed in the coding lecture!

The data should now be fully preprocessed. We will use the `scderp` object for cell identification in the next assignment.